<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_make_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#KcBERT 학습용 데이터셋을 제작하는 코드(로컬)
#제작할 데이터셋은 pretraining용 데이터셋(soynlp 학습용 데이터), finetuning용 데이터셋(next sentence prediction, text classification)

In [ ]:
!pip install kss
!pip install datasets
!pip install soynlp
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
from soynlp.normalizer import *
from kss import Kss
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction, TrainingArguments, BertForSequenceClassification
from datasets import Dataset, load_dataset, ClassLabel
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
#전처리된 파일 로드(BP_spacing 건너뛰기)
talk_save_path = '/content/drive/My Drive/talk_preprocess_result_short.xlsx'
qna_path = '/content/drive/My Drive/community_qna_preprocessed.xlsx'
all_contents_path = '/content/drive/My Drive/community_all_contents_preprocessed.xlsx'
all_comments_path = '/content/drive/My Drive/community_all_comments_preprocessed.xlsx'

df_talk = pd.read_excel(talk_save_path)
df_qna = pd.read_excel(qna_path)
df_all_contents = pd.read_excel(all_contents_path)
df_all_comments = pd.read_excel(all_comments_path)

In [ ]:
qna_path = '/content/drive/My Drive/community_qna_preprocessed.xlsx'
df_qna = pd.read_excel(qna_path)

In [ ]:
#직접 제작한 카카오톡 질문답변 데이터셋 로드
talk_finetune_path = '/content/drive/My Drive/talk_finetune_dataset.xlsx'

df_talk_finetune = pd.read_excel(talk_finetune_path, index_col = 0)

In [ ]:
#직접 제작한 카카오톡 질문 데이터셋 로드
talk_question_finetune_path = '/content/drive/My Drive/talk_finetune_question_dataset.xlsx'

df_talk_finetune_question = pd.read_excel(talk_question_finetune_path, index_col = 0)

#Soyspacing 학습시킨 후에 띄어쓰기 교정

In [ ]:
#일단 보류

#Pretrain 데이터셋, 제공받은 모든 데이터 사용


In [ ]:
split_sentences = Kss('split_sentences')

In [ ]:
#kss로 리스트 안에 있는 여러 문장으로 이루어진 글을 문장 단위로 분리하는 함수
def separate_to_sentences_and_deduplicate(DataFrame, columns):
    sentences_list = []
    for column in columns: #데이터프레임의 각 열에 대해서 처리
        text_list = DataFrame[column].to_list()
        sentences_list.append(split_sentences(text_list))
    return list(set(sum([lists for lists in sentences_list[0]], [])))

In [ ]:
#데이터프레임의 열들을 리스트로 변환
talk_list = separate_to_sentences_and_deduplicate(df_talk, ['text'])


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
#데이터프레임의 NaN 대신 '' 채워넣기
df_qna.fillna('', inplace = True)
df_all_contents.fillna('', inplace = True)
df_all_comments.fillna('', inplace = True)

In [ ]:
qna_list = separate_to_sentences_and_deduplicate(df_qna, ['내용', '답변', '댓글'])

In [ ]:
all_contents_list = separate_to_sentences_and_deduplicate(df_all_contents, ['내용', '답변', '댓글'])

In [ ]:
all_comments_list = separate_to_sentences_and_deduplicate(df_all_comments, ['내용', '대댓글', '댓글'])
print(all_comments_list)

In [ ]:
#하나의 리스트로 결합
result_list = talk_list + qna_list + all_contents_list + all_comments_list
print(result_list[:10])
print(len(result_list))
df_result = pd.DataFrame(result_list, columns = ['text'])
print(df_result)
print(len(df_result))

In [ ]:
#전체 데이터에 대해 soynlp.normalize 적용?
#일단 보류

In [ ]:
#.csv 파일로 저장(.xlsx로 저장 시 illegalcharactererror 발생)
pretrain_data_save_path = '/content/drive/My Drive/KcBERT_pretrain_dataset_short.csv'

df_result.to_csv(pretrain_data_save_path)

#Finetune 데이터셋(질문-답변)


In [ ]:
#slug가 있다면 ['내용'] == 질문
df_question_answer = df_qna[pd.isna(df_qna['Slug']) == False]
df_question_answer = df_question_answer[pd.isna(df_question_answer['답변']) == False]

print(df_question_answer)
print(len(df_question_answer))

print(df_question_answer.iloc[0])
print(df_question_answer.iloc[1]['답변'])

질문 + 답변에다가 답변에 달린 댓글들까지 입력 데이터로 처리할 경우 퀄리티가 더 좋아질 수 있지만, max_length를 초과하지 않는 방향에 따라 답변 데이터만 이용, 나중에 개선방안 찾아보기

community_qna_preprocessed 데이터에서 이미지 파일을 base64 인코딩한 텍스트가 섞여 있는데, 데이터 파일 추출 과정에서 직접 제거하는 게 좋을 것 같음.

In [ ]:
#커뮤니티 파일을 가공해서 finetune 데이터셋을 제작
finetune_dict = {}

#slug가 있다면 ['내용'] == 질문
df_question_answer = df_qna[df_qna['Slug'] != '']
df_question_answer = df_question_answer[df_question_answer['답변'] != '']

print(df_question_answer.iloc[2])

#['답변']의 텍스트를 ' , ' 로 구분 후 질문에 하나씩 쌓기
sent1_list, sent2_list, label_list, index_list = [], [], [], []
for i in range(len(df_question_answer)):
  row = df_question_answer.iloc[i]
  question_text = row['내용']
  answer_text_list = row['답변'].split(' , ')
  for j in range(len(answer_text_list)):
    if answer_text_list[j] != '':
      sent1_list.append(question_text + ''.join(answer_text_list[:j]))
      sent2_list.append(answer_text_list[j])
      label_list.append(1)
      index_list.append(i)

finetune_dict = {'sent1' : sent1_list, 'sent2' : sent2_list, 'label' : label_list, 'sent2_index' : index_list}

df_community_finetune = pd.DataFrame.from_dict(finetune_dict)
print(df_community_finetune)

In [ ]:
#파인튜닝 데이터셋에서 n배의 negative sample을 생성하는 코드
def create_negative_sample(DataFrame, n):
  DataFrame_copy = DataFrame.copy() #원본 데이터셋을 변화시키지 않기 위해 deep copy
  length = len(DataFrame_copy)
  for i in range(length):
    row = DataFrame_copy.iloc[i]
    negative_candidate_ids = list(set(DataFrame_copy['sent2_index'].tolist())) #.remove(row['sent2_index']) #해당 행의 sent2를 negative sample 후보군에서 제외
    negative_candidate_ids.remove(row['sent2_index'])
    negative_sample_ids = list(np.random.choice(negative_candidate_ids, n, replace = False))
    for idx in negative_sample_ids:
      new_row = pd.Series({'sent1' : row['sent1'], 'sent2' : np.random.choice(DataFrame_copy[DataFrame_copy['sent2_index'] == idx]['sent2'].tolist(), 1)[0], 'label' : 0, 'sent2_index' : idx}) #카톡 데이터는 idx 중복이 없기 때문에 랜덤추출로 1개를 뽑아도 달라지는 건 없고, 커뮤니티 데이터의 경우 idx 중복이 있기 때문에 해당 idx를 가진 sent2 중 하나를 뽑아야 함.
      DataFrame_copy = pd.concat([DataFrame_copy, new_row.to_frame().T], ignore_index = True)
  return DataFrame_copy

In [ ]:
#커뮤니티 파인튜닝 데이터셋에서 negative sample 생성
df_community_finetune_negative_sampled = create_negative_sample(df_community_finetune, 7)
# df_community_finetune_negative_sampled = df_community_finetune_negative_sampled.drop('sent2_index', axis = 1)
print(len(df_community_finetune_negative_sampled))
print(df_community_finetune_negative_sampled)

In [ ]:
print(df_community_finetune_negative_sampled[df_community_finetune_negative_sampled['sent2'] == ''])
print(df_community_finetune[df_community_finetune['sent2_index'] == 13])

In [ ]:
print(df_talk_finetune)

In [ ]:
#카카오톡 파인튜닝 데이터셋에서 negative sample 생성
df_talk_finetune_negative_sampled = create_negative_sample(df_talk_finetune, 7)
df_talk_finetune_negative_sampled = df_talk_finetune_negative_sampled.drop('sent2_index', axis = 1)
print(len(df_talk_finetune_negative_sampled))
print(df_talk_finetune_negative_sampled)

In [ ]:
print(df_talk_finetune_negative_sampled[df_talk_finetune_negative_sampled['sent2'] == ''])

Empty DataFrame
Columns: [sent1, sent2, label]
Index: []


In [ ]:
#두 파인튜닝 데이터셋을 하나로 합치기
df_KcBERT_finetune = pd.concat([df_community_finetune_negative_sampled, df_talk_finetune_negative_sampled], ignore_index = True)
print(len(df_KcBERT_finetune))

5824


In [ ]:
#.xlsx 파일로 저장
finetune_data_save_path = '/content/drive/My Drive/KcBERT_finetune_dataset_answer.xlsx'

df_KcBERT_finetune.to_excel(finetune_data_save_path)

#Finetune 데이터셋(질문)

왜 빌더로그 게시글 요소들을 세 부분으로 나눠야 하는가?
1. positive sample, negative sample 비율 1대1이 되도록 만들기
2. 사용할 RobertaForSequenceClassification 아키텍처를 보면 (position_embeddings): Embedding(514, 768, padding_idx=1)이 포함되어 있다. negative sample에 해당하는 빌더로그 게시글만 길이가 길다면 이러한 정보까지 학습할 여지가 있다.

왜 tokenizer()로 바로 시퀀스 길이를 구하지 않고 원본 리스트부터 시작해서 평균 길이를 구하는가
->

인자 중 max_length가 존재하기 때문에 어차피 길이를 초과하는 데이터는 truncation이 진행된다.

그렇기 때문에 데이터 평균 길이를 비교하려면 원본 리스트를 사용해야 한다.

In [ ]:
#토큰화를 위한 pretrained model 로드
pretrained_model_path = '/content/drive/My Drive/Pretrained_Model'

pretrained_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path)
pretrained_model = BertForSequenceClassification.from_pretrained(pretrained_model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/Pretrained_Model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#제작한 데이터셋의 결측치 제거
df_talk_finetune_question = df_talk_finetune_question[(pd.notna(df_talk_finetune_question['text'])) & (pd.notna(df_talk_finetune_question['label']))]
print(df_talk_finetune_question)
df_talk_finetune_question_positive = df_talk_finetune_question[df_talk_finetune_question['label'] == 1]
print(df_talk_finetune_question_positive)

In [ ]:
#데이터셋의 tokenized data의 길이를 구하는 함수
def tokenized_sequence_length(text):
  #kss 모듈로 글을 문장으로 분리
  original_sentences = split_sentences(text)
  index_sentences = pretrained_tokenizer(original_sentences)['input_ids']
  index_sentences_filtered = [sentence[1:-1] for sentence in index_sentences] #cls_token과 sep_token 제외
  total_length = sum(len(s) for s in index_sentences_filtered)
  return index_sentences_filtered, total_length

In [ ]:
#데이터셋에서 각 요소를 n등분하는 함수
def divide_text_to_number(text, number):
  #kss 모듈로 문장으로 분리
  index_sentences, total_length = tokenized_sequence_length(text)
  target_length = total_length / number

  # 세 부분으로 분할
  segments = []
  current_segment = []
  current_length = 0

  #마지막 세그먼트 추가를 위해 사용하는 인덱스 값
  sentence_checkpoint = 0

  for index, sentence in enumerate(index_sentences):
      if current_length + len(sentence) > target_length and current_segment:
          segments.append(sum(current_segment, []))
          current_segment = [sentence]
          current_length = len(sentence)
          if len(segments) == number - 1:  # 이미 (목표 세그먼트 개수 - 1)개 세그먼트를 만들었다면 나머지는 모두 마지막 세그먼트에 추가
              sentence_checkpoint = index #current_segment에 추가해둔 문장을 인덱싱
              break
      else:
          current_segment.append(sentence)
          current_length += len(sentence)

  # 마지막 세그먼트 추가
  segments.append(sum(current_segment + index_sentences[sentence_checkpoint + 1:], []))

  return pretrained_tokenizer.batch_decode(segments)


In [ ]:
#BP_judge_question용 데이터 제작

#slug가 있다면 질문, 없다면 답변, 이 중 질문을 positive sample로 설정
question_list = df_qna[pd.isna(df_qna['Slug']) == False]['내용'].dropna(axis = 0).to_list()
label_question = [1] * len(question_list)

print(question_list)
print('데이터셋의 전체 길이 :', len(question_list))

#질문 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(question) for question in question_list) / len(question_list))

#질문 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in question_list) / len(question_list))

In [ ]:
#답변을 negative sample로 설정
answer_list = df_qna['답변'].dropna(axis = 0).to_list()
label_answer = [0] * len(answer_list)

print(answer_list)
print(len(answer_list))

#답변 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(answer) for answer in answer_list) / len(answer_list))

#답변 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in answer_list) / len(answer_list))

#시퀀스 길이 균형을 위해 각 답변 글을 n등분
answer_list_divided = sum([divide_text_to_number(text, 2) for text in answer_list], [])

#분할한 데이터의 개수에 맞춰서 레이블 설정
label_answer = [0] * (len(answer_list_divided))

#분리된 답변 글 평균 길이 확인(토큰화 결과)
print('분리된 데이터셋에서 토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in answer_list_divided) / len(answer_list_divided))

#분리된 데이터셋의 전체 길이
print('분리된 데이터셋의 전체 길이 :', len(answer_list_divided))

In [ ]:
#빌더로그 글을 negative sample로 설정
builder_log_list = df_all_contents[df_all_contents['커뮤니티 타입'] == '빌더 로그']['내용'].dropna(axis = 0).to_list()

print(builder_log_list)
print(len(builder_log_list))

#빌더로그 글 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(builder_log) for builder_log in builder_log_list) / len(builder_log_list))

#빌더로그 글 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in builder_log_list) / len(builder_log_list))

#시퀀스 길이 균형을 위해 각 빌더로그 글을 n등분
builder_log_list_divided = sum([divide_text_to_number(text, 6) for text in builder_log_list], [])

#분할한 데이터의 개수에 맞춰서 레이블 설정
label_builder_log = [0] * (len(builder_log_list_divided))

#분리된 빌더로그 글 평균 길이 확인(토큰화 결과)
print('분리된 데이터셋에서 토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in builder_log_list_divided) / len(builder_log_list_divided))

#분리된 데이터셋의 전체 길이
print('분리된 데이터셋의 전체 길이 :', len(builder_log_list_divided))

In [ ]:
#커뮤니티 글 데이터프레임 생성
text = question_list + answer_list + builder_log_list_divided
label = label_question + label_answer + label_builder_log

df_judge_question = pd.DataFrame([text, label], index = ['text', 'label'])
df_judge_question = df_judge_question.T
df_judge_question = df_judge_question[pd.isna(df_judge_question['text']) == False]
print(df_judge_question)


In [ ]:
#직접 제작한 카카오톡 데이터프레임과 결합
df_KcBERT_finetune_question = pd.concat([df_judge_question, df_talk_finetune_question], ignore_index = True)
print(df_KcBERT_finetune_question)

In [ ]:
#.xlsx 파일로 저장
finetune_data_save_path = '/content/drive/My Drive/KcBERT_finetune_dataset_question.xlsx'

df_KcBERT_finetune_question.to_excel(finetune_data_save_path)